In [129]:
import pandas as pd
import numpy as np
import seaborn as sns
import re
from nltk.corpus import stopwords
import codecs
import nltk
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from ydata_profiling import ProfileReport


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\moren\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


segmentar los el nivel de ventas según precio, idioma, nivel y rating de cada curso para analizar qué tanto influyen dichas variables en la demanda del producto vendido.

WordCloud de las palabras clave que más se repiten dentro del título. (Se puede añadir otras variables de nuestro interés).


Por último, se nos pide una demo en un rango de tiempo de no más de 10 min donde presentamos las funcionalidades del dashboard y las conclusiones/recomendaciones de nuestra parte.

Con el fin de monitorear la eficacia de los objetivos de la empresa, se le pide establecer al menos 1 KPI producto de su análisis y que el mismo se pueda visualizar en un dashboard.


In [66]:
coursera  = pd.read_csv('Coursera_courses.csv')
c_reviews = pd.read_csv('Coursera_reviews.csv')
edx = pd.read_csv('edx_courses.csv')
udemy = pd.read_csv('udemy_courses.csv')

In [80]:
coursera.head()


,name,institution,course_url,course_id,rating,course_id_count
0,Machine Learning,Stanford University,https://www.coursera.org/learn/machine-learning,machine-learning,4.750522,35895
1,Indigenous Canada,University of Alberta,https://www.coursera.org/learn/indigenous-canada,indigenous-canada,4.813614,3702
2,The Science of Well-Being,Yale University,https://www.coursera.org/learn/the-science-of-...,the-science-of-well-being,4.928404,20085
3,Technical Support Fundamentals,Google,https://www.coursera.org/learn/technical-suppo...,technical-support-fundamentals,4.776183,35319
4,Become a CBRS Certified Professional Installer...,Google - Spectrum Sharing,https://www.coursera.org/learn/google-cbrs-cpi...,google-cbrs-cpi-training,4.931034,87


rating, institution

In [ ]:
c_reviews.head()



In [ ]:
df_rating = c_reviews.groupby('course_id')['rating'].mean().reset_index()
coursera = pd.merge(coursera, df_rating, on='course_id')

In [ ]:
temp_df = c_reviews['course_id'].value_counts().reset_index()
temp_df.columns = ['course_id', 'course_id_count']
coursera = coursera.merge(temp_df, on='course_id', how='left')

In [79]:
edx.head()


,title,n_enrolled,course_type,institution,instructors,Level,subject,language,subtitles,price,course_length_weeks,min_effort,max_effort
0,How to Learn Online,"124,980",Self-paced on your time,edX,Nina Huntemann-Robyn Belair-Ben Piscopo,Introductory,Education & Teacher Training,English,English,49.0,2,2,3
1,Programming for Everybody (Getting Started wit...,"293,864",Self-paced on your time,The University of Michigan,Charles Severance,Introductory,Computer Science,English,English,49.0,7,2,4
2,CS50's Introduction to Computer Science,"2,442,271",Self-paced on your time,Harvard University,David J. Malan-Doug Lloyd-Brian Yu,Introductory,Computer Science,English,English,90.0,12,6,18
3,The Analytics Edge,"129,555",Instructor-led on a course schedule,Massachusetts Institute of Technology,Dimitris Bertsimas-Allison O'Hair-John Silberh...,Intermediate,Data Analysis & Statistics,English,English,199.0,13,10,15
4,Marketing Analytics: Marketing Measurement Str...,"81,140",Self-paced on your time,"University of California, Berkeley",Stephan Sorger,Introductory,Computer Science,English,English,249.0,4,5,7


n_enrolled, course_type, institution, instructors, level, subject, language, subtitles, price, course_length_weeks, min_effort, max_effort
n_enrolled

In [67]:
edx = edx.drop(columns=["course_url", "course_syllabus", "course_description", "summary"])

In [68]:
def extract_price(price):
    match = re.search(r'\$(\d+(\.\d+)?)', price)
    if match:
        return float(match.group(1))
    else:
        return 0.0

edx['price'] = edx['price'].apply(extract_price)

In [69]:

def extract_length(length):
    match = re.search(r'(\d+)', length)
    if match:
        return int(match.group(1))
    else:
        return None

edx['course_length_weeks'] = edx['course_length'].apply(lambda x: extract_length(x))


In [71]:
def extract_effort(text):
    try:
        min_effort, max_effort = map(int, re.findall(r'\d+', text))
        return min_effort, max_effort
    except:
        return None, None


edx[['min_effort', 'max_effort']] = edx['course_effort'].apply(extract_effort).apply(pd.Series)
edx = edx.drop(['course_length'], axis=1)

In [72]:
edx = edx.drop(['course_effort'], axis=1)

In [81]:
udemy.head()

,course_title,is_paid,price,num_subscribers,num_reviews,num_lectures,level,content_duration,subject
0,Ultimate Investment Banking Course,True,200,2147,23,51,All Levels,1.5,Business Finance
1,Complete GST Course & Certification - Grow You...,True,75,2792,923,274,All Levels,39.0,Business Finance
2,Financial Modeling for Business Analysts and C...,True,45,2174,74,51,Intermediate Level,2.5,Business Finance
3,Beginner to Pro - Financial Analysis in Excel ...,True,95,2451,11,36,All Levels,3.0,Business Finance
4,How To Maximize Your Profits Trading Options,True,200,1276,45,26,Intermediate Level,2.0,Business Finance


is_paid, price, num_reviews, num_lectures, level, content_duration, subject
num_suscribers

In [76]:
profile_coursera = ProfileReport(coursera, title="courseraReport")
profile_review = ProfileReport(c_reviews, title="reviewReport", minimal=True)
profile_edx = ProfileReport(edx, title="edxReport")
profile_udemy = ProfileReport(udemy, title="udemyReport")

In [89]:
profile_coursera



Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
profile_review



In [ ]:
profile_edx


In [77]:

profile_udemy

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [75]:
udemy = udemy.drop(['course_id','url','published_timestamp'], axis=1)

In [92]:
wordcloud_df = pd.concat([udemy[["course_title"]], edx[["title"]], coursera[["name"]]], axis=1)
wordcloud_df = wordcloud_df.rename(columns={'course_title':'udemy', 'title':'edx', 'name':'coursera'})

wordcloud_df = pd.melt(wordcloud_df, value_vars=["udemy", "edx", "coursera"], var_name="source", value_name="course_name")
wordcloud_df = wordcloud_df.dropna()

In [122]:
udemy["course_title"].to_csv("udemy_txt.txt", index=False, header=False)
coursera["name"].to_csv("coursera_txt.txt", index=False, header=False)
edx["title"].to_csv("edx_txt.txt", index=False, header=False)
wordcloud_df["course_name"].to_csv("nube_txt.txt", index=False, header=False)

In [130]:
with codecs.open("udemy_txt.txt", "r",encoding="ISO-8859-1") as file:
    udemy_txt = file.read()
with codecs.open("coursera_txt.txt", "r",encoding="ISO-8859-1") as file:
    coursera_txt = file.read()
with codecs.open("edx_txt.txt", "r",encoding="ISO-8859-1") as file:
    edx_txt = file.read()
with codecs.open("nube_txt.txt", "r",encoding="ISO-8859-1") as file:
    nube_txt = file.read()

# Generar la wordcloud
udemy_wc = WordCloud(background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(udemy_txt)
    
coursera_wc = WordCloud(background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(coursera_txt)
edx_wc = WordCloud(background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(edx_txt)
all_wc = WordCloud(background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(nube_txt)

# Mostrar la wordcloud
plt.imshow(udemy_wc, interpolation='bilinear')
plt.axis("off")
plt.show()
# Mostrar la wordcloud
plt.imshow(coursera_wc, interpolation='bilinear')
plt.axis("off")
plt.show()
# Mostrar la wordcloud
plt.imshow(edx_wc, interpolation='bilinear')
plt.axis("off")
plt.show()
# Mostrar la wordcloud
plt.imshow(all_wc, interpolation='bilinear')
plt.axis("off")
plt.show()

C:\Users\moren\AppData\Local\Temp\ipykernel_18164\1809217909.py:28: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\moren\AppData\Local\Temp\ipykernel_18164\1809217909.py:32: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\moren\AppData\Local\Temp\ipykernel_18164\1809217909.py:36: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()
C:\Users\moren\AppData\Local\Temp\ipykernel_18164\1809217909.py:40: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [119]:


def create_wordcloud(df, column_name, title):
    # Convertir la columna especificada a una sola cadena de texto
    text = " ".join(review for review in df[column_name])
    
    # Crear la nube de palabras
    wordcloud = WordCloud(width = 800, height = 800, 
                background_color ='white', 
                stopwords = stopwords, 
                min_font_size = 10).generate(text)
    
    # Mostrar la nube de palabras
    plt.figure(figsize = (8, 8), facecolor = None) 
    plt.imshow(wordcloud) 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title)
    plt.show()

In [120]:
create_wordcloud(udemy, ["course_title"], "Tendencias UDEMY")


TypeError: 'WordListCorpusReader' object is not iterable

In [114]:
create_wordcloud(coursera, ["name"], "Tendencias EDX")


TypeError: 'WordListCorpusReader' object is not iterable

In [115]:
create_wordcloud(edx, ["title"], "Tendencias Coursera")


TypeError: 'WordListCorpusReader' object is not iterable

In [116]:
create_wordcloud(wordcloud_df, ["course_name"], "Tendencias Generales")

TypeError: 'WordListCorpusReader' object is not iterable

In [78]:
coursera.to_csv('coursera_clean.csv')
edx.to_csv('edx_clean.csv')
udemy.to_csv('udemy_clean.csv')
